In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import json

In [2]:
min_threshold = 0.5
quorum = 25

In [3]:
with open("nominees.json") as f:
    nominees = json.load(f)

president_nominees = nominees["president"]
vp_nominees = nominees["vp"]
avp_nominees = nominees["avp"]
trez_nominees = nominees["trez"]

In [4]:
members = pd.read_csv("members.csv")
member_ids = members["uid"].to_list()

In [5]:
def get_ballot_ranked_candidates(votes : pd.Series, candidates : list[str], eliminated_candidates : list[str] = []) -> list[str] | None:
    if candidates is None or len(candidates) == 0:
        return []
    if not votes.dropna().is_unique:
        return None
    votes = votes.apply(lambda x : int(x.split()[1]) if pd.notna(x) else x)
    number_of_ranked_candidates = len(votes.dropna())
    if np.intersect1d(votes.dropna().values, range(1, number_of_ranked_candidates + 1)).size != number_of_ranked_candidates:
        return None
    votes = votes.reset_index()
    votes = votes.sort_values(ascending=True, by=votes.columns[-1])
    candidates = [candidate for candidate in candidates if candidate not in eliminated_candidates]
    return [candidates[index] for index in votes.dropna().index]

In [6]:
def parse_absentee(raw_df : pd.DataFrame) -> pd.DataFrame:
    president_start_col = 5
    vp_start_col = president_start_col + len(president_nominees)
    avp_start_col = vp_start_col + len(vp_nominees)
    trez_start_col = avp_start_col + len(avp_nominees)
    trez_end_col = trez_start_col + len(trez_nominees)

    raw_df["President_Ranked"] = raw_df.apply(lambda row : get_ballot_ranked_candidates(row[president_start_col:vp_start_col], president_nominees), axis=1)
    raw_df["VP_Ranked"] = raw_df.apply(lambda row : get_ballot_ranked_candidates(row[vp_start_col:avp_start_col], vp_nominees), axis=1)
    raw_df["AVP_Ranked"] = raw_df.apply(lambda row : get_ballot_ranked_candidates(row[avp_start_col:trez_start_col], avp_nominees), axis=1)
    raw_df["Trez_Ranked"] = raw_df.apply(lambda row : get_ballot_ranked_candidates(row[trez_start_col:trez_end_col], trez_nominees), axis=1)

    df = raw_df.loc[:, ["Email", "President_Ranked", "VP_Ranked", "AVP_Ranked", "Trez_Ranked"]].set_index("Email")
    return df

In [7]:
def parse_vote(raw_live_df : pd.DataFrame, absentee_df : pd.DataFrame, candidates : list[str], column : str, eliminated_candidates : list[str] = []) -> pd.DataFrame:
    raw_live_df[column] = raw_live_df.apply(lambda row : get_ballot_ranked_candidates(row[5:], candidates, eliminated_candidates), axis=1)
    df = raw_live_df.loc[:, ["Email", column]].set_index("Email")
    df = pd.concat([absentee_df.loc[:, [column]], df])
    df.index = df.index.str.split("@").str[0]
    return df

In [17]:
def irv_process(df : pd.DataFrame, candidates : list[str], eliminated_candidates : list[str]) -> (str, int, int):
    df = df.copy(deep=True)
    total_ballots = len(df)

    absentee_revotes = df.index.duplicated(keep="last").sum()
    df = df[~df.index.duplicated(keep="last")]

    num_non_member_votes = df.index.difference(member_ids).size

    df = df.loc[member_ids]

    spoiled_votes = df.iloc[:,0].isna().sum(numeric_only=True)
    number_df = df.dropna().apply(lambda row : len(row.iloc[0]), axis = 1)
    abstained_votes = number_df[number_df == 0].count()

    total_voters = len(df)

    if total_voters < quorum:
        raise Exception("Quorum not met, total voters: " + str(total_voters), ", must wait for more votes")

    current_candidates = [candidate for candidate in candidates if candidate not in eliminated_candidates]
    candidate_ballots = {candidate: [] for candidate in current_candidates}

    candidate_ranked_votes = pd.DataFrame(0, columns=range(1, len(candidates) + 1), index=candidates)

    for index, row in df.dropna().iterrows():
        ballot = row.iloc[0].copy()
        for index, val in enumerate(ballot):
            candidate_ranked_votes.loc[val, index + 1] += 1
        while len(ballot) > 0 and ballot[0] not in current_candidates:
            ballot.pop(0)
        if len(ballot) == 0:
            continue
        candidate_ballots[ballot[0]].append(ballot)

    winner = None
    winner_votes = -1
    ballot_rounds = []

    while candidate_ballots:
        lowest_votes = float("inf")
        lowest_candidates = []
        current_ballot_round = {}

        highest_votes = -1
        highest_candidate = None

        for candidate in candidate_ballots.keys():
            candidate_votes = len(candidate_ballots[candidate])
            current_ballot_round[candidate] = candidate_votes

            if candidate_votes < lowest_votes:
                lowest_votes = candidate_votes
                lowest_candidates = [candidate]
            elif candidate_votes == lowest_votes:
                lowest_candidates.append(candidate)

            if candidate_votes > highest_votes:
                highest_votes = candidate_votes
                highest_candidate = candidate

        ballot_rounds.append(current_ballot_round)

        if highest_votes > total_voters * min_threshold:
            winner = highest_candidate
            winner_votes = highest_votes
            break

        for candidate in lowest_candidates:
            for ballot in candidate_ballots[candidate]:
                while len(ballot) > 0 and (ballot[0] in lowest_candidates or ballot[0] not in candidate_ballots.keys()):
                    ballot.pop(0)
                if len(ballot) > 0:
                    candidate_ballots[ballot[0]].append(ballot)
            
            candidate_ballots.pop(candidate)

    candidate_ranked_votes = candidate_ranked_votes.reset_index().melt(id_vars=["index"], value_name="Votes", var_name="Rank").rename(columns={"index" : "Nominee"})

    spoiled_abstained_df = pd.DataFrame({"Ballot Type" : ["Spoiled", "Abstained", "Absentee Revote (Voided Absentee Vote)"], "Votes" : [spoiled_votes, abstained_votes, absentee_revotes]})

    fig, axs = plt.subplots(2 + len(ballot_rounds), figsize=(15, 15), constrained_layout=True)

    sb.catplot(data=candidate_ranked_votes, x="Nominee", y="Votes", hue="Rank", kind="bar")
    sb.barplot(data={
        "Ballot Type" : ["Member Votes", "Non-Member Votes"],
        "Votes" : [total_voters, num_non_member_votes]
    }, x="Ballot Type", y="Votes", ax=axs[0])
    sb.barplot(data=spoiled_abstained_df, x="Ballot Type", y="Votes", ax=axs[1])

    for i in range(len(ballot_rounds)):
        sb.barplot(data=pd.DataFrame.from_dict(ballot_rounds[i], orient="index", columns=["Votes"]).reset_index(), x="index", y="Votes", ax=axs[i + 2])
        axs[i + 2].set_xlabel("Candidates")
        axs[i + 2].set_title("Round " + str(i + 1))
    
    for ax in axs:
        ax.grid(visible=True, axis="y")

    if winner:
        return (winner, winner_votes, total_ballots)

    raise Exception("No winner")

In [18]:
raw_absentee = pd.read_excel('absentee.xlsx')
absentee = parse_absentee(raw_absentee)

In [19]:
raw_prez = pd.read_excel('president.xlsx')
prez = parse_vote(raw_prez, absentee, president_nominees, "President_Ranked")

In [ ]:
elected_prez, president_number_of_votes, total_number_of_ballots = irv_process(prez, president_nominees, [])
print("President: " + elected_prez + " with " + str(president_number_of_votes) + " votes out of " + str(total_number_of_ballots) + " ballots")

In [103]:
raw_vp = pd.read_excel('vp.xlsx')
vp = parse_vote(raw_vp, absentee, vp_nominees, "VP_Ranked", [elected_prez])

In [ ]:
elected_vp, vp_number_of_votes, total_number_of_ballots = irv_process(vp, vp_nominees, [elected_prez])
print("VP: " + elected_vp + " with " + str(vp_number_of_votes) + " votes out of " + str(total_number_of_ballots) + " ballots")

In [112]:
raw_avp = pd.read_excel('avp.xlsx')
avp = parse_vote(raw_avp, absentee, avp_nominees, "AVP_Ranked", [elected_prez, elected_vp])

In [ ]:
elected_avp, avp_number_of_votes, total_number_of_ballots = irv_process(avp, avp_nominees, [elected_prez, elected_vp])
print("AVP: " + elected_avp + " with " + str(avp_number_of_votes) + " votes out of " + str(total_number_of_ballots) + " ballots")

In [114]:
raw_trez = pd.read_excel('trez.xlsx')
trez = parse_vote(raw_trez, absentee, trez_nominees, "Trez_Ranked", [elected_prez, elected_vp, elected_avp])

In [ ]:
elected_trez, trez_number_of_votes, total_number_of_ballots = irv_process(trez, trez_nominees, [elected_prez, elected_vp, elected_avp])
print("Trez: " + elected_trez + " with " + str(trez_number_of_votes) + " votes out of " + str(total_number_of_ballots) + " ballots")

In [ ]:
print("Elected: " + elected_prez + " (President), " + elected_vp + " (VP), " + elected_avp + " (AVP), " + elected_trez + " (Trez)")